# Init

In [1]:
# data_dir
DATA_DIR = "./data"

# create connection
library(RPostgres)
wrds <- dbConnect(Postgres(),
                  host='wrds-pgdata.wharton.upenn.edu',
                  port=9737,
                  dbname='wrds',
                  sslmode='require',
                  user='xiaomowu',
                  password='SLCyz2018')

# WRDS

## ind-profile

402,835 obs

Contains variables such as `gender`, `dob`, `nationality`...

In [8]:
library = 'boardex'
dataset = 'na_wrds_dir_profile_details' # individual profiles

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_profiles <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_prof)

-individual_profiles- saved  (3.55 secs)


## ind-edu

1,341,622 obs

In [9]:
library = 'boardex'
dataset = 'na_dir_profile_education' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_edu <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_edu)

-individual_edu- saved  (6.68 secs)


## ind-emp

6,302,551 obs

Contains "which persons have served in the focal company"

In [12]:
library = 'boardex'
dataset = 'na_wrds_dir_profile_emp' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_emp <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_emp)

-individual_emp- saved  (47.63 secs)


## firm-profile

919,051 obs

Which contains `firm-id` and its `ticker`

In [2]:
library = 'boardex'
dataset = 'na_wrds_company_profile' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_prof_from_boardex <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_prof_from_boardex)

-firm_prof_from_boardex- saved  (6.06 secs)


In [ ]:
library = 'crspa'
dataset = 'msf' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_prof_from_crsp <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_prof_from_crsp)

## firm-net

3,631,160 obs

In [4]:
library = 'boardex'
dataset = 'na_wrds_company_networks' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
firm_net <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(firm_net)

-firm_net- saved  (38.16 secs)


## ind-net (dep)

Didn't use, because it's too large (806,646,080 obs) 

In [ ]:
library = 'boardex'
dataset = 'na_wrds_individual_networks' # individual education

res <- dbSendQuery(wrds, sprintf("select * from %s.%s", library, dataset))
individual_net <- dbFetch(res, n=-1) %>% setDT()
dbClearResult(res)
sv(individual_net)

# Clean

## ind_prof

Table `f_ind_prof`
- The profiles of the individuals that we may use. Each person has one exact observation. Note that we may not use all of them.
- For demo of `f_ind_prof` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `dob`: date of birth
- `dod`: date of death
- `age`: the age given by the database. Seems to be calcualted by 2015.
- `gender`: the gender of the individual
- `nationality`: nationality of the individual

In [1]:
ld(individual_prof, force=T)

-individual_prof- loaded  (1.74 secs)


In [4]:
f_ind_prof = individual_prof[, 
    .(name=directorname, name_id=directorid, 
      dob=dob,
      dod=dod, 
      age=as.integer(age)+4, gender=gender,
      nationality=nationality)] %>%
    unique()
f_ind_prof[1]
fwrite(f_ind_prof, './data/f_ind_prof.csv')

name,name_id,dob,dod,age,gender,nationality
David Shaw,16,22 Apr 1951,n.a.,68,M,American


## ind_edu

Table `f_ind_edu`
- Education of the individuals that we may use. Each person may have MULTIPLE observations, if they receive education from multiple institutions. Note that we may not use all of them.
- For demo of `f_ind_edu` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `edu_type`: type of education. Could be "Education" or "Professional Qualifications"
- `university`: the institution the individual receive education from. 
- `university_id`: the unique id for each `university`
- `qualification`: the type of qualificatoin, e.g. BA, PhD... 
- `award_date`: the date to be awarded the qualification.

In [6]:
ld(individual_edu, force=T)
f_ind_edu = individual_edu[,
    .(name=directorname, name_id=directorid, edu_type=rowtype,
      university=companyname, university_id=companyid,
      qualification=qualification, award_date=awarddate)] %>%
    unique()
f_ind_edu[1]
fwrite(f_ind_edu, './data/f_ind_edu.csv')

-individual_edu- loaded  (1.64 secs)


name,name_id,edu_type,university,university_id,qualification,award_date
David Shaw,16,Education,University of New Hampshire,62520,BA,1973-01-01


## ind_emp

Table `f_ind_emp`
- Employment history of each individual. One individual may have MULTIPLE observations, if he/she has served in multiple companies.
- For demo of `f_ind_emp` please see the end of this section
- `name`: the name of the individual
- `name_id`: unique id for that individual
- `company`: the name of the company that the individual has served in.
- `company_id`: unique id for each `company`.
- `in_board`: "Yes" if the individual was in the "board of directors", otherwise "No".
- `in_supervisory`: "Yes" if the individual was in the "supervisory board", otherwise "No". 
- `role`: the role of the indiviudal when he/she was in the company
- `role_start_date`: the date when the individual started his/her role
- `role_end_date`: the date when the individual ended his/her role.

In [7]:
ld(individual_emp, force=T)

-individual_emp- loaded  (20.55 secs)


In [8]:
f_ind_emp = individual_emp[,
    .(name=directorname, name_id=directorid,
      company=companyname, company_id=companyid, 
      in_board=brdposition, in_supervisory=ned, role=rolename,
      role_start_date=datestartrole, role_end_date=dateendrole)] %>%
    unique()
fwrite(f_ind_emp, './data/f_ind_emp.csv')
# sv(f_ind_emp)
f_ind_emp[1]

name,name_id,company,company_id,in_board,in_supervisory,role,role_start_date,role_end_date
Ambassador Gerald Carmen,372547,Home Shopping Latino Inc (1 800 Autotow Inc prior to 11/2006),2,Yes,Yes,Director - SD,1998-07-01,NA


## firm_prof

Table `f_firm_prof`
- Profiles for the companies that we may use. Each company has exactly one observation
- For demo of `f_firm_prof` please see the end of this section
- `company`: the name of the company
- `company_id`: unique id for each `company`.
- `ticker`: ticker of the company, e.g. "MSFT" for Microsoft

In [49]:
ld(firm_prof_from_boardex, force=T)

-firm_prof_from_boardex- loaded  (3.46 secs)


In [57]:
cols = c('hoaddress1', 'hoaddress2', 'hoaddress3', 'hoaddress4', 'hoaddress5', 'hocountryname', 'ccaddress1', 'ccaddress2', 'ccaddress3', 'ccaddress4', 'ccaddress5', 'cccountryname')

f_firm_prof = firm_prof_from_boardex[
    countryofquote=='U.S.A.' & hocountryname=='United States'
    ][, c(cols) := lapply(.SD, str_replace_na, ''), .SDcols=cols
    ][, .(company=boardname, company_id=boardid, sector,
      headoffice_address=str_c(
        hoaddress1, hoaddress2, hoaddress3,
        hoaddress4, hoaddress5, hocountryname, sep=' '),
      communication_address=str_c(
        ccaddress1, ccaddress2, ccaddress3,
        ccaddress4, ccaddress5, cccountryname, sep=' '),
      market_cap=mktcapitalisation,
      n_employee=noemployees,
      revenue)
    ][order(-market_cap)] %>% unique()
fwrite(f_firm_prof, './data/f_firm_prof.csv')
sv(f_firm_prof)
f_firm_prof[1]

-f_firm_prof- saved  (0.07 secs)


company,company_id,sector,headoffice_address,communication_address,market_cap,n_employee,revenue
MICROSOFT CORP,20601,Software & Computer Services,One Microsoft Way Redmond Washington WA 98052-6399 United States,One Microsoft Way Redmond Washington WA 98052-6399 United States,1060781,131000,110360


In [54]:
f_firm_prof[order(-market_cap)][1:10]

company,company_id,sector,headoffice_address,communication_address,market_cap,n_employee,revenue
MICROSOFT CORP,20601,Software & Computer Services,One Microsoft Way Redmond Washington WA 98052-6399 United States,One Microsoft Way Redmond Washington WA 98052-6399 United States,1060781,131000,110360
AMAZON.COM INC,1703,General Retailers,410 Terry Avenue North Seattle Washington WA 98109-5210 United States,410 Terry Avenue North Seattle Washington WA 98109-5210 United States,894229,647500,232887
APPLE INC (Apple Computer Inc prior to 01/2007),2355,Information Technology Hardware,One Apple Park Way Cupertino California CA 95014 United States,1 Infinite Loop Cupertino California CA 95014 United States,888113,132000,265595
ALPHABET INC (Google Inc prior to 10/2015),116351,Software & Computer Services,1600 Amphitheatre Parkway Mountain View California CA 94043 United States,1600 Amphitheatre Parkway Mountain View California CA 94043 United States,776176,98771,136819
BERKSHIRE HATHAWAY INC,4080,Insurance,3555 Farnam Street Omaha Nebraska NE 68131 United States,3555 Farnam Street Omaha 68131 United States,491060,389000,247837
FACEBOOK INC,1820060,Software & Computer Services,1601 Willow Road Menlo Park California CA 94025 United States,1601 Willow Road Menlo Park California CA 94025 United States,457557,35587,55838
JPMORGAN CHASE & CO,17528,Banks,383 Madison Avenue New York NY 10179-0001 United States,383 Madison Avenue 10179-0001 United States,354223,256105,109029
JOHNSON & JOHNSON,17478,Pharmaceuticals and Biotechnology,One Johnson & Johnson Plaza New Brunswick New Jersey NJ 08933 United States,One Johnson & Johnson Plaza New Brunswick New Jersey NJ 08933 United States,343247,135100,81581
WALMART INC (Wal-Mart Stores Inc prior to 02/2018),33046,General Retailers,702 Southwest 8th Street Bentonville Arkansas AR 72716-0215 United States,702 Southwest 8th Street Bentonville Arkansas AR 72716 United States,330613,2200000,514405
EXXON MOBIL CORP,11430,Oil & Gas,5959 Las Colinas Boulevard Irving Texas TX 75039-2298 United States,5959 Las Colinas Boulevard Irving Texas TX 75039-2298 United States,306311,71100,290212


In [38]:
f_firm_prof[, table(sector)] %>% as.data.table() 

sector,N
Aerospace & Defence,57
Automobiles & Parts,77
Banks,854
Beverages,31
Blank Check / Shell Companies,123
Business Services,382
Chemicals,178
Clothing & Personal Products,87
Construction & Building Materials,150
Consumer Services,42


## firm_net

Table `f_firm_net`
- Network between firms. Perhapse the most import table in our analysis.
- Each line stands for one link from the `from_company` to the `to_company`
- Definition of company link: if one person has served both in company A and company B (not necessarily at the same time), then we say a link has be established from A to B through the person.
- For demo of `f_firm_net` please see the end of this section
- `from_company`: the start company of the link.
- `from_company_id`: the unque id for the `from_company`
- `to_company`: the end company of the link.
- `to_company_id`: the unique id for the `to_company`. 
- `name`: the name of the individual that established the link
- `name_id`: unique id for `name`.
- `role_in_from_company`: the role of the individual when he/she is in the `from_company`.
- `role_in_from_company_start_date`: the date when the individual started his/her role in the `from_company` 
- `role_in_from_company_end_date`: the date when the individual ended his/her role in the `from_company` 
- `role_in_to_company`: the role of the individual when he/she is in the `to_company`.
- `role_in_to_company_start_date`: the date when the individual started his/her role in the `to_company` 
- `role_in_to_company_end_date`: the date when the individual ended his/her role in the `to_company`

In [11]:
ld(firm_net, force=T)
f_firm_net = firm_net[,
    .(from_company=boardname, from_company_id=boardid,
      to_company=companyname, to_company_id=companyid,
      name=directorname, name_id=directorid,
      role_in_from_company=role,
      role_in_from_company_start_date=startcompanydatestartrole,
      role_in_from_company_end_date=startcompanydateendrole,
      role_in_to_company=associatedrole,
      role_in_to_company_start_date=conncompanydatestartrole,
      role_in_to_company_end_date=conncompanydateendrole)] %>%
    unique()
fwrite(f_firm_net, './data/f_firm_net.csv')
# sv(f_firm_net)
f_firm_net[1]

-firm_net- loaded  (11.14 secs)


from_company,from_company_id,to_company,to_company_id,name,name_id,role_in_from_company,role_in_from_company_start_date,role_in_from_company_end_date,role_in_to_company,role_in_to_company_start_date,role_in_to_company_end_date
GREENHILL & CO INC,40757,1 800 CONTACTS INC (De-listed 09/2007),3,Steve Key,59796,Independent Director (Brd) (SD),2004-05-01,NA,Independent Director (Brd) (SD),2005-07-28,2007-09-07
